# Normalizador - Experimento

Este é um componente que normaliza as amostras individualmente de acordo com a norma da unidade. Faz uso da implementação do [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para montagem de modelo, pré-processamento de dados, seleção e avaliação de modelos, e muitos outros utilitários.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
# parâmetros
dataset = "/tmp/data/iris.csv" #@param {type:"string"}
target = "Species" #@param {type:"feature", label:"Atributo alvo", description: "Seu modelo será treinado para prever os valores do alvo."}

norm = "l2" #@param ["l1", "l2", "max"] {type:"string", label:"Norma", description:"A norma a ser usada para normalizar cada amostra diferente de zero. Se a norma 'max' for usada, os valores serão redimensionados pelo máximo dos valores absolutos"}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd

df = pd.read_csv(dataset)
X = df.drop(target, axis=1)
y = df[target]

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Configuração dos atributos

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Selects the indexes of numerical
numerical_indexes = np.where(featuretypes == NUMERICAL)[0]
non_numerical_indexes = np.where(~(featuretypes == NUMERICAL))[0]

# After the step of the make_column_transformer,
# numerical features are grouped in the beggining of the array
numerical_indexes_after_first_step = np.arange(len(numerical_indexes))

## Treina um modelo usando sklearn.preprocessing.Normalizer

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

pipeline = make_pipeline(
    make_column_transformer(
        (SimpleImputer(), numerical_indexes), remainder="passthrough"
    ),
    make_column_transformer(
        (Normalizer(norm=norm), numerical_indexes_after_first_step),
        remainder="passthrough",
    ),
)

# Train model and transform dataset
X = pipeline.fit_transform(X)

# Put numerical features in the lowest indexes
features_after_pipeline = np.concatenate(
    (columns[numerical_indexes], columns[non_numerical_indexes])
)

# Put data back in a pandas.DataFrame
df = pd.DataFrame(data=X, columns=features_after_pipeline)
df[target] = y

## Salva alterações no conjunto de dados

O conjunto de dados será salvo (e sobrescrito com as respectivas mudanças) localmente, no container da experimentação, utilizando a função `pandas.DataFrame.to_csv`.<br>

In [ ]:
# save dataset changes
df.to_csv(dataset, index=False)

## Salva modelo e outros resultados do treinamento

Escreve todos artefatos na pasta `/tmp/data/`. A plataforma guarda os artefatos desta pasta para usos futuros como implantação e comparação de resultados.

In [ ]:
from joblib import dump

artifacts = {
    "pipeline": pipeline,
    "columns": columns,
    "features_after_pipeline": features_after_pipeline,
}

dump(artifacts, "/tmp/data/normalizer.joblib")